# **13.2 Try_Except_Blocks**

In the real world, things go wrong — players type invalid levels, network requests fail, save files get corrupted. The `try/except` block is Python's way of letting you **anticipate** these problems and handle them gracefully instead of crashing your program. In this lesson you'll learn how to wrap risky code, catch exceptions, and keep your Pokemon game running smoothly even when something unexpected happens.

---

## **The Basic try/except Structure**

The `try` block contains code that *might* fail. If it does, Python jumps straight to the `except` block instead of crashing. Code after a successful `try` block continues normally.

In [ ]:
# Without try/except — program crashes on bad input
# user_input = "Pikachu"
# level = int(user_input)   # Crashes! ValueError

# With try/except — program handles it gracefully
user_input = "Pikachu"   # Imagine this came from the player

try:
    level = int(user_input)          # This might fail
    print(f"Pokemon level: {level}") # Only runs if int() succeeds
except:
    print(f"'{user_input}' is not a valid level — please enter a number")

print("Game continues...")

---

## **How it Works Step by Step**

Understanding the flow through a try/except block makes it much easier to write them correctly.

In [ ]:
def demo_flow(value):
    """Show exactly which lines execute for a given input."""
    print(f"\n--- Testing with value: {value!r} ---")
    print("1. Before try block")
    
    try:
        print("2. Inside try — about to convert")
        result = int(value)           # Might fail here
        print(f"3. Conversion succeeded: {result}")  # SKIPPED if error
        print("4. End of try block")                  # SKIPPED if error
    except:
        print("3. EXCEPTION CAUGHT — inside except")  # Only runs on error
    
    print("5. After try/except — always runs")

demo_flow("25")       # Success path: lines 1,2,3,4,5
demo_flow("Pikachu")  # Error path:   lines 1,2,3(except),5

---

## **Catching the Exception Object**

Using `except Exception as e` gives you access to the actual exception object so you can read its message, log it, or display it to help with debugging.

In [ ]:
# Catch the exception object for more detail
inputs = ["25", "Pikachu", "3.7", "-5", "100"]

for raw in inputs:
    try:
        level = int(raw)
        if level < 1 or level > 100:
            raise ValueError(f"Level must be 1–100, got {level}")
        print(f"  ✓ Valid level: {level}")
    except ValueError as e:
        # 'e' contains the exception details
        print(f"  ✗ Invalid input '{raw}': {e}")

---

## **Bare except vs except Exception**

You have two options for catching all errors: `except:` (bare) catches absolutely everything including system exits, while `except Exception:` catches all normal errors but leaves system events alone. Always prefer `except Exception` — bare `except` can mask serious problems.

In [ ]:
# AVOID — bare except catches EVERYTHING, including KeyboardInterrupt
# try:
#     ...risky code...
# except:                # Too broad — hides serious problems
#     print("Error")

# PREFER — except Exception catches all normal errors but not system events
def load_pokemon_data(filename):
    """Load Pokemon data from a file, handling all normal errors."""
    try:
        with open(filename) as f:
            return f.read()
    except Exception as e:
        print(f"Failed to load '{filename}': {e}")
        return None

data = load_pokemon_data("pikachu_save.json")  # File doesn't exist
data2 = load_pokemon_data("/dev/null")          # Exists but empty

print(f"Data 1: {data}")
print(f"Data 2: '{data2}'")

---

## **Nested try/except**

You can nest try/except blocks to handle errors at different levels of your code — outer blocks handle broad problems, inner blocks handle specific operations.

In [ ]:
# Nested try/except for a Pokemon battle simulator
import random

def simulate_turn(attacker, defender):
    """Simulate one battle turn with nested error handling."""
    try:
        # Outer try: handles general battle errors
        if not attacker or not defender:
            raise ValueError("Both attacker and defender must be provided")
        
        try:
            # Inner try: handles the damage calculation specifically
            damage = attacker['attack'] * random.randint(85, 100) // 100
            defender['hp'] -= damage
            print(f"{attacker['name']} deals {damage} damage!")
            print(f"{defender['name']} HP: {max(0, defender['hp'])}")
        except KeyError as e:
            print(f"Battle data missing key: {e}")
            
    except ValueError as e:
        print(f"Battle setup error: {e}")

pikachu = {"name": "Pikachu",  "attack": 55, "hp": 35}
onix    = {"name": "Onix",     "attack": 45, "hp": 70}

simulate_turn(pikachu, onix)
simulate_turn(None, onix)              # ValueError
simulate_turn({"name": "Ghost"}, onix) # KeyError — missing 'attack'

---

## **Re-raising Exceptions**

Sometimes you want to log an error or clean up resources, but still let the error propagate upward to the caller. Use a bare `raise` inside the except block to re-raise the same exception.

In [ ]:
import datetime

error_log = []

def log_and_reraise(operation, func, *args):
    """Run a function, log any error, then re-raise it."""
    try:
        return func(*args)
    except Exception as e:
        # Log the error for later review
        entry = {
            'time': str(datetime.datetime.now()),
            'operation': operation,
            'error': str(e)
        }
        error_log.append(entry)
        print(f"[LOG] Error in '{operation}': {e}")
        raise   # Re-raise the SAME exception — caller decides what to do

# Outer handler catches the re-raised exception
try:
    log_and_reraise("parse_level", int, "Pikachu")
except ValueError:
    print("Outer handler: recovered from the ValueError")

print(f"\nError log: {error_log}")

---

## **try/except with Return Values**

A common real-world pattern is to return a sentinel value (like `None` or a default) from the `except` block, indicating that the operation failed cleanly.

In [ ]:
def safe_parse_level(raw: str) -> int | None:
    """
    Parse a level string, returning None if invalid.
    Caller checks the return value instead of catching exceptions.
    """
    try:
        level = int(raw)
        if level < 1 or level > 100:
            return None    # Valid int but out of range
        return level
    except ValueError:
        return None        # Not a valid integer at all

test_inputs = ["25", "0", "150", "Pikachu", "36"]

for raw in test_inputs:
    level = safe_parse_level(raw)
    if level is None:
        print(f"  '{raw}' → invalid level")
    else:
        print(f"  '{raw}' → Level {level}")

---

## **Practical Pokemon Example**

In [ ]:
# A robust Pokemon catch attempt that handles all failure modes
import random

POKEDEX = {
    "Pikachu":  {"catch_rate": 190, "hp": 35},
    "Charizard":{"catch_rate": 45,  "hp": 78},
    "Mewtwo":   {"catch_rate": 3,   "hp": 106},
}

def attempt_catch(pokemon_name: str, pokeball_type: str = "regular") -> str:
    """
    Attempt to catch a Pokemon.
    Returns a result string describing what happened.
    """
    try:
        # Look up the Pokemon — raises KeyError if not found
        data = POKEDEX[pokemon_name]
        
        # Determine catch multiplier
        multipliers = {"regular": 1, "great": 1.5, "ultra": 2, "master": 999}
        if pokeball_type not in multipliers:
            raise ValueError(f"Unknown ball type: '{pokeball_type}'")
        
        ball_mult = multipliers[pokeball_type]
        
        # Roll the catch
        catch_chance = (data['catch_rate'] * ball_mult) / 255
        success = random.random() < catch_chance
        
        if success:
            return f"{pokemon_name} was caught! 🎉"
        else:
            return f"{pokemon_name} broke free!"
    
    except KeyError:
        return f"Error: '{pokemon_name}' is not in the Pokedex"
    except ValueError as e:
        return f"Error: {e}"

# Test various scenarios
print(attempt_catch("Pikachu"))
print(attempt_catch("Mewtwo", "ultra"))
print(attempt_catch("Eevee"))               # Not in POKEDEX
print(attempt_catch("Charizard", "super"))  # Invalid ball

---

## **Practice Exercises**

### **Task 1: Basic try/except**

Wrap the conversion in a try/except so the program doesn't crash.

**Expected Output:**
```
'Pikachu' is not a valid number
```

In [ ]:
user_input = "Pikachu"

# Your code here:


### **Task 2: Catch with Message**

Catch the exception object and print its message.

**Expected Output:**
```
Error: invalid literal for int() with base 10: 'thirty'
```

In [ ]:
raw = "thirty"

# Your code here:


### **Task 3: Return None on Failure**

Write `safe_int(s)` that returns an int or None.

**Expected Output:**
```
25
None
```

In [ ]:
# Your code here:


### **Task 4: Safe Dictionary Lookup**

Wrap a dict access in try/except to handle missing keys.

**Expected Output:**
```
Pikachu found: Electric
Eevee not in Pokedex
```

In [ ]:
pokedex = {"Pikachu": "Electric", "Charizard": "Fire"}

for name in ["Pikachu", "Eevee"]:
    # Your try/except here:
    pass

### **Task 5: Safe Division**

Handle ZeroDivisionError when calculating win rate.

**Expected Output:**
```
Win rate: 0.0%
```

In [ ]:
wins = 0
battles = 0

# Your code here:


### **Task 6: Multiple Except**

Handle both ValueError and TypeError separately.

**Expected Output:**
```
25
ValueError: 'Pikachu' is not numeric
TypeError: got NoneType, expected str
```

In [ ]:
def parse_level(raw):
    # Handle ValueError for bad strings and TypeError for None/wrong types
    pass

# Your code here:


### **Task 7: Always Continue**

Loop through inputs, using try/except so bad values are skipped and the loop always completes.

**Expected Output:**
```
Level: 25
Skipped: Pikachu
Level: 36
Skipped: high
Level: 8
```

In [ ]:
inputs = ["25", "Pikachu", "36", "high", "8"]

# Your code here:


### **Task 8: Log Errors**

Collect all errors in a list as the loop runs.

**Expected Output:**
```
Valid: [25, 36, 8]
Errors: ['Pikachu', 'abc']
```

In [ ]:
inputs = ["25", "Pikachu", "36", "abc", "8"]

# Your code here:


### **Task 9: Safe Catch Attempt**

Write `safe_catch(name)` that looks up a Pokemon and returns a result string.

**Expected Output:**
```
Attempting Pikachu...
Attempting Eevee...
Eevee is not in the Pokedex
```

In [ ]:
pokedex = {"Pikachu": 190, "Charizard": 45, "Mewtwo": 3}

# Your code here:


### **Task 10: Re-raise**

Log the error, then re-raise it so the outer handler can deal with it.

**Expected Output:**
```
[LOG] Cannot convert 'Pikachu' to int
Outer handler caught it!
```

In [ ]:
# Your code here:


---

## **Summary**

- `try:` wraps code that might raise an exception
- `except:` runs only when the `try` block fails
- Code after a successful `try` block is skipped on error
- Code after the whole `try/except` always continues
- `except Exception as e:` gives you access to the error object
- Prefer `except Exception` over bare `except:`
- Return `None` or a default to signal clean failure
- Use bare `raise` to re-raise after logging
- Nest try/except blocks for layered error handling

---

## **Quick Reference**

```python
# Basic structure
try:
    risky_operation()
except Exception as e:
    print(f"Error: {e}")

# Return None on failure
def safe_op(x):
    try:
        return int(x)
    except ValueError:
        return None

# Re-raise after logging
try:
    ...
except Exception as e:
    log(e)
    raise        # re-raises same exception

# Skip bad items in a loop
for item in items:
    try:
        process(item)
    except Exception:
        continue
```